# Goal statement

need to predict a players total points in the tourney as a function of their season points total, their avg points per game, and Tournament Seed

## Data 

below we are trying to ensure we can get:
- the players name, 
- the school they play for
-  conference the school is in
- point per game average
- total points for season

In [14]:
import pandas as pd
from fuzzywuzzy import fuzz
import os

In [107]:


def get_season_year(file):
    y = os.path.basename(file).split('.')[0].split('_')[-1].split('-')[1]
    return f'20{y}'


def split_player_name(row):

    player = row['Player']
    
    array = player.split(',')
    c = len(array)
    print_it = False
    pn: str = ''
    school: str = ''
    conf: str = ''

    if c == 2:
        pn = array[0]
        a = array[1].split('(')
    elif c == 3:
        pn = f'{array[0]} {array[1].strip()}'
        a = array[2].split('(')
        print_it = True
    else:
        raise KeyError(f'ERROR: Could not determine Name, school and conference {player}')
    
    if len(a) == 2:
        school = a[0]
        conf = a[1]
        
    elif len(a) == 3:
        conf = a[2]
        school = f'{a[0].strip()} {a[1].strip()}'.replace(')', '')
        
    else:
        raise KeyError(f'ERROR: Could not determine school and conference {a}')
    
    conf = conf.replace(')','')
    return pn.strip(), school.strip(), conf.strip()
  

def read_data(file):
    df = pd.read_csv(file, dtype=str, index_col='Rank')
    df['Year'] = get_season_year(file)
    df[['Name','School','Conf']] = df.apply(split_player_name, axis=1, result_type='expand')

    return df


player_stats = read_data('/workspaces/ncaa-player-pool/data/prediction/ncaa_statistics_2022-23.csv')

display(player_stats)






,Player,Cl,Ht,Pos,G,FGM,3FG,FT,PTS,PPG,Year,Name,School,Conf
Rank,,,,,,,,,,,,,,
1,"Antoine Davis, Detroit Mercy (Horizon)",Sr.,6-1,G,33,303,159,165,930,28.2,2023,Antoine Davis,Detroit Mercy,Horizon
4,"Darius McGhee, Liberty (ASUN)",Sr.,5-9,G,33,249,153,99,750,22.7,2023,Darius McGhee,Liberty,ASUN
7,"Taevion Kinsey, Marshall (Sun Belt)",Sr.,6-5,G,32,271,19,145,706,22.1,2023,Taevion Kinsey,Marshall,Sun Belt
15,"Drew Pember, UNC Asheville (Big South)",Sr.,6-10,F,33,192,55,252,691,20.9,2023,Drew Pember,UNC Asheville,Big South
8,"Max Abmas, Oral Roberts (Summit League)",Sr.,6-0,G,31,216,111,139,682,22,2023,Max Abmas,Oral Roberts,Summit League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,"Jaykwon Walton, Wichita St. (AAC)",Jr.,6-7,G,26,127,36,75,365,14,2023,Jaykwon Walton,Wichita St.,AAC
300,"Briggs McClain, Hartford (DI Independent)",Jr.,6-4,G,25,139,39,44,361,14.4,2023,Briggs McClain,Hartford,DI Independent
211,"Dawson Garcia, Minnesota (Big Ten)",So.,6-11,F,23,132,25,66,355,15.4,2023,Dawson Garcia,Minnesota,Big Ten


Next need to get the player_stats for year merged with ranking and tourney seed data


In [148]:
def get_schools(ranking_data):
    return ranking_data.School.unique()

def match_schools(school: str, schools):
    found_school: bool = False
    print(school)
    if school in schools:
        print(f'FOUND: {school}')
        found_school = True
    else:
        for s in schools:
            if fuzz.ratio(school, s) >= .9:
                print(f'{school} ==> {s}')
    return found_school
        
def code_dump(): 
    schools = get_schools(ranking_data=rank_df)

    ps_index = player_stats.set_index('School').index
    ps_year_index = player_stats.set_index('Year').index
    schools_index = rank_df.set_index('School').index
    schools_year_index = rank_df.set_index('Year').index

    display(schools_year_index)

    display(player_stats[~ps_index.isin(schools_index)])

rank_file = '/workspaces/ncaa-player-pool/data/prediction/archive/538 Ratings.csv'

rank_df = pd.read_csv(rank_file, dtype=str, header=0, index_col='TEAM NO')

rank_df = rank_df.rename(columns={'YEAR': 'Year', 'TEAM': 'School'})

d23 = ( 
    player_stats.merge(rank_df,
                      on=['Year', 'School'],
                      how='inner', 
                      indicator=True)
        .drop(columns='_merge')
    )
display(d23)


,Player,Cl,Ht,Pos,G,FGM,3FG,FT,PTS,PPG,Year,Name,School,Conf,SEED,ROUND,POWER RATING,POWER RATING RANK
0,"Drew Pember, UNC Asheville (Big South)",Sr.,6-10,F,33,192,55,252,691,20.9,2023,Drew Pember,UNC Asheville,Big South,15,64,73,427
1,"Max Abmas, Oral Roberts (Summit League)",Sr.,6-0,G,31,216,111,139,682,22,2023,Max Abmas,Oral Roberts,Summit League,12,64,79.2,343
2,"Drew Timme, Gonzaga (WCC)",Sr.,6-10,F,31,259,2,134,654,21.1,2023,Drew Timme,Gonzaga,WCC,3,8,89.9,46
3,"Zach Edey, Purdue (Big Ten)",Jr.,7-4,C,29,247,0,147,641,22.1,2023,Zach Edey,Purdue,Big Ten,1,64,89.5,50
4,"Kendric Davis, Memphis (AAC)",Sr.,5-11,G,29,191,48,188,618,21.3,2023,Kendric Davis,Memphis,AAC,8,64,84.9,164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,"Norchad Omier, Miami (FL) (ACC)",So.,6-7,F,30,164,5,89,422,14.1,2023,Norchad Omier,Miami FL,ACC,5,4,83.6,210
63,"Ryan Kalkbrenner, Creighton (Big East)",Jr.,7-1,C,28,171,4,76,422,15.1,2023,Ryan Kalkbrenner,Creighton,Big East,6,8,87.6,92
64,"Seth Lundy, Penn St. (Big Ten)",Sr.,6-6,F,29,135,79,65,414,14.3,2023,Seth Lundy,Penn St.,Big Ten,10,32,82.8,240
65,"LJ Cryer, Baylor (Big 12)",Jr.,6-1,G,28,141,72,56,410,14.6,2023,LJ Cryer,Baylor,Big 12,3,32,87.1,104


next need clean up tourney points data

In [149]:
tourney_stats_file = '/workspaces/ncaa-player-pool/data/prediction/ncaa_statistics_champ_2022-23.csv'

tourney_df = pd.read_csv(tourney_stats_file, dtype=str, header=0, index_col='Rank').filter(items=['PTS', 'Player'])

tourney_df['Year'] = get_season_year(tourney_stats_file)
tourney_df = tourney_df.rename(columns={'PTS': 'TourneyPTS'})
#tourney_df[['Name','School','Conf']] = tourney_df.apply(split_player_name, axis=1, result_type='expand')
display(tourney_df.sort_values(axis=0, by='Rank',ascending=True))

d23_f = ( 
    d23.merge(tourney_df,
            on=['Year', 'Player'],
            how='inner', 
            indicator=True)
        .drop(columns='_merge')
    )

display(d23_f)

,TourneyPTS,Player,Year
Rank,,,
1,32,"Terquavion Smith, NC State (ACC)",2023
2,27,"Kedrian Johnson, West Virginia (Big 12)",2023
2,27,"RaeQuan Battle, Montana St. (Big Sky)",2023
4,50,"Mike Miles, TCU (Big 12)",2023
5,97,"Drew Timme, Gonzaga (WCC)",2023
...,...,...,...
346,10,"Alonzo Gaffney, Arizona St. (Pac-12)",2023
346,10,"Trey Galloway, Indiana (Big Ten)",2023
346,10,"Garrett Hien, Furman (SoCon)",2023


,Player,Cl,Ht,Pos,G,FGM,3FG,FT,PTS,PPG,Year,Name,School,Conf,SEED,ROUND,POWER RATING,POWER RATING RANK,TourneyPTS
0,"Drew Pember, UNC Asheville (Big South)",Sr.,6-10,F,33,192,55,252,691,20.9,2023,Drew Pember,UNC Asheville,Big South,15,64,73,427,13
1,"Max Abmas, Oral Roberts (Summit League)",Sr.,6-0,G,31,216,111,139,682,22,2023,Max Abmas,Oral Roberts,Summit League,12,64,79.2,343,12
2,"Drew Timme, Gonzaga (WCC)",Sr.,6-10,F,31,259,2,134,654,21.1,2023,Drew Timme,Gonzaga,WCC,3,8,89.9,46,97
3,"Zach Edey, Purdue (Big Ten)",Jr.,7-4,C,29,247,0,147,641,22.1,2023,Zach Edey,Purdue,Big Ten,1,64,89.5,50,21
4,"Kendric Davis, Memphis (AAC)",Sr.,5-11,G,29,191,48,188,618,21.3,2023,Kendric Davis,Memphis,AAC,8,64,84.9,164,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,"Norchad Omier, Miami (FL) (ACC)",So.,6-7,F,30,164,5,89,422,14.1,2023,Norchad Omier,Miami FL,ACC,5,4,83.6,210,50
61,"Ryan Kalkbrenner, Creighton (Big East)",Jr.,7-1,C,28,171,4,76,422,15.1,2023,Ryan Kalkbrenner,Creighton,Big East,6,8,87.6,92,80
62,"Seth Lundy, Penn St. (Big Ten)",Sr.,6-6,F,29,135,79,65,414,14.3,2023,Seth Lundy,Penn St.,Big Ten,10,32,82.8,240,21
63,"LJ Cryer, Baylor (Big 12)",Jr.,6-1,G,28,141,72,56,410,14.6,2023,LJ Cryer,Baylor,Big 12,3,32,87.1,104,45


In [126]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
X = [[ 1,  2,  3],  # 2 samples, 3 features
     [11, 12, 13]]
y = [0, 1]  # classes of each sample
clf.fit(X, y)

a = clf.predict([[4, 5, 6], [14, 15, 16]])
print(a)


df = pd.DataFrame([[1, 2.12], [3.356, 4.567]])
df
df.map(lambda x: len(str(x)))


SyntaxError: invalid syntax (2688125364.py, line 17)